In [1]:
!pip install twitter
!pip install feedparser
!pip install schedule
!pip install kafka-python
!pip install redis
!pip install uuid

In [2]:
from time import sleep
from json import dumps, loads, load
from kafka import KafkaProducer
from datetime import datetime, timedelta
from pytz import timezone
from feedparser import parse
from urllib.request import urlopen
from twitter import Twitter
import schedule
from datetime import datetime
import uuid

In [11]:
tickers = ["MSFT","AAPL"]
feeds = ["http://feeds.reuters.com/reuters/businessNews",
         "https://finance.yahoo.com/news/rssindex/", 
         "https://www.nasdaq.com/feed/rssoutbound",
         "https://news.microsoft.com/feed",
        "https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml",
        "https://feeds.a.dj.com/rss/RSSMarketsMain.xml",
        "http://www.marketwatch.com/rss/topstories",
        "http://www.marketwatch.com/rss/marketpulse",
        "http://www.marketwatch.com/rss/realtimeheadlines",
        "http://feeds.marketwatch.com/marketwatch/bulletins",
        "https://www.cnbc.com/id/10000664/device/rss/rss.html",
        "https://www.cnbc.com/id/10001147/device/rss/rss.html",
        "http://feeds.feedburner.com/TechCrunch/",
        "https://www.ft.com/?format=rss",
        "https://rss.nytimes.com/services/xml/rss/nyt/Business.xml",
        "https://abcnews.go.com/abcnews/moneyheadlines",
        "https://www.cbsnews.com/latest/rss/main",
        "https://markets.businessinsider.com/rss/news",
        "http://newsrss.bbc.co.uk/rss/newsonline_uk_edition/front_page/rss.xml",
        "http://feeds.bbci.co.uk/news/business/rss.xml",
        ]
yfeeds = []

producer = KafkaProducer(
    bootstrap_servers=['broker:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

# Authenticate with Twitter & search
# https://pypi.org/project/twitter/
# https://developer.twitter.com/en/apps/17019807
from twitter import *
t = Twitter(auth=OAuth(consumer_key="nDwKHutvaskyGecZkCS1SiYY8", consumer_secret="zlQdJIuFUGGglbMO2kYMk6oNLFztzipxhRANNz5q5gqfn1tGb1", token="73568030-GqfoqkfWRhqhARS9uzz51vg1UGSV7AGRaOxDkAurZ", token_secret="1kiHGfyDfO7gcaJbGFnr3lnLA3vXc8ti9hJhj1lnTTvEm"))

In [12]:
last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
import pandas as pd
feed_data = {}
for feed in feeds:
    try:
        feed_data[feed] = [{"Type": "News", "Timestamp": datetime.timestamp(pd.to_datetime(l.published)), "Title":l.title, "Summary":l.summary, "Link":l.link} for l in (parse(feed))["entries"] if pd.to_datetime(l.published )>last_time_step]
    except TypeError:
        print("TypeError")

In [13]:
for feed in feeds:
    print(f'{len(feed_data[feed])} from {feed}')

20 from http://feeds.reuters.com/reuters/businessNews
50 from https://finance.yahoo.com/news/rssindex/
15 from https://www.nasdaq.com/feed/rssoutbound
2 from https://news.microsoft.com/feed
18 from https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml
20 from https://feeds.a.dj.com/rss/RSSMarketsMain.xml
9 from http://www.marketwatch.com/rss/topstories
30 from http://www.marketwatch.com/rss/marketpulse
10 from http://www.marketwatch.com/rss/realtimeheadlines
10 from http://feeds.marketwatch.com/marketwatch/bulletins
20 from https://www.cnbc.com/id/10000664/device/rss/rss.html
29 from https://www.cnbc.com/id/10001147/device/rss/rss.html
20 from http://feeds.feedburner.com/TechCrunch/
16 from https://www.ft.com/?format=rss
42 from https://rss.nytimes.com/services/xml/rss/nyt/Business.xml
25 from https://abcnews.go.com/abcnews/moneyheadlines
50 from https://www.cbsnews.com/latest/rss/main
10 from https://markets.businessinsider.com/rss/news
43 from http://newsrss.bbc.co.uk/rss/newsonline_uk_edi

In [21]:
parse("https://finance.yahoo.com/news/rssindex/")

{'feed': {'title': 'Yahoo Finance',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://finance.yahoo.com/news/rssindex/',
   'value': 'Yahoo Finance'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://finance.yahoo.com/'}],
  'link': 'https://finance.yahoo.com/',
  'subtitle': 'At Yahoo Finance, you get free stock quotes, up-to-date news, portfolio management resources, international market data, social interaction and mortgage rates that help you manage your financial life.',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://finance.yahoo.com/news/rssindex/',
   'value': 'At Yahoo Finance, you get free stock quotes, up-to-date news, portfolio management resources, international market data, social interaction and mortgage rates that help you manage your financial life.'},
  'language': 'en-US',
  'rights': 'Copyright (c) 2019 Yahoo! Inc. All rights reserved',
  'rights_detail': {'type': 'tex

In [25]:
feed_data["https://news.microsoft.com/feed"]

[{'Type': 'News',
  'Timestamp': 1576868416.0,
  'Title': 'Microsoft and KKBOX Group launch global strategic partnership',
  'Summary': '<p>TAIPEI — December 20, 2019 — Microsoft Taiwan and Asia’s leading media technology company, KKBOX Group, jointly announced today the launch of a global strategic partnership that will migrate the group’s subsidiary KKBOX’s music streaming services to the Microsoft Azure cloud platform. Additionally, KKBOX Group subsidiary KKStream has joined Microsoft’s global partner network to release <span aria-label="ellipsis">[…]</span></p>\n<p>The post <a rel="nofollow" href="https://news.microsoft.com/2019/12/20/microsoft-and-kkbox-group-launch-global-strategic-partnership/">Microsoft and KKBOX Group launch global strategic partnership</a> appeared first on <a rel="nofollow" href="https://news.microsoft.com">Stories</a>.</p>',
  'Link': 'https://news.microsoft.com/2019/12/20/microsoft-and-kkbox-group-launch-global-strategic-partnership/'},
 {'Type': 'News',
 

In [6]:
len([entry['Title'] for feed in feeds for entry in feed_data[feed]])

133

In [7]:
output = set()
for x in [entry['Summary'] for feed in feeds for entry in feed_data[feed]]:
    output.add(x)
print(len(output))

131


In [18]:
print(list(output)[1].encode("utf8"))

b'Britain has approved the purchase of British defense company Cobham  by U.S. investor Advent International for $5 billion after the private equity group made commitments to address national security concerns.<div class="feedflare">\n<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=1u0qnwpZWWk:-1yc_poI31E:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=1u0qnwpZWWk:-1yc_poI31E:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=1u0qnwpZWWk:-1yc_poI31E:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=1u0qnwpZWWk:-1yc_poI31E:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=1u0qnwpZWWk:-1yc_poI31E:V_sGLiPBpWU" border="0"></img></a>\n</div><img src="http://feeds.feedburner.com/~r/reuters/businessNews/~4/1u0qnwpZWWk" height="1" width="1" alt=""/>'


In [16]:
str(list(output)[1].encode("utf8"))

'b\'Britain has approved the purchase of British defense company Cobham  by U.S. investor Advent International for $5 billion after the private equity group made commitments to address national security concerns.<div class="feedflare">\\n<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=1u0qnwpZWWk:-1yc_poI31E:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=1u0qnwpZWWk:-1yc_poI31E:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=1u0qnwpZWWk:-1yc_poI31E:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=1u0qnwpZWWk:-1yc_poI31E:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=1u0qnwpZWWk:-1yc_poI31E:V_sGLiPBpWU" border="0"></img></a>\\n</div><img src="http://feeds.feedburner.com/~r/reuters/businessNews/~4/1u0qnwpZWWk" height="1" width="1" alt=""/>\''

In [14]:
import base64
print(str(list(output)[2].encode("unicode_escape").decode("utf8")).encode("utf8")) #.decode("unicode_escape"))

b'The Boeing Co Starliner spacecraft that failed in its mission to reach the International Space Station was due to barrel down to the Earth\'s surface early on Sunday with the daunting task of landing safely.<div class="feedflare">\\n<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=Nm3TKS0f5Ww:EE4xQwCTaD4:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=Nm3TKS0f5Ww:EE4xQwCTaD4:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=Nm3TKS0f5Ww:EE4xQwCTaD4:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=Nm3TKS0f5Ww:EE4xQwCTaD4:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=Nm3TKS0f5Ww:EE4xQwCTaD4:V_sGLiPBpWU" border="0"></img></a>\\n</div><img src="http://feeds.feedburner.com/~r/reuters/businessNews/~4/Nm3TKS0f5Ww" height="1" width="1" alt=""/>'


In [13]:
print(str(list(output)[2].encode("unicode_escape").decode("utf8")).encode("utf8").decode("unicode_escape"))

The Boeing Co Starliner spacecraft that failed in its mission to reach the International Space Station was due to barrel down to the Earth's surface early on Sunday with the daunting task of landing safely.<div class="feedflare">
<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=Nm3TKS0f5Ww:EE4xQwCTaD4:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=Nm3TKS0f5Ww:EE4xQwCTaD4:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=Nm3TKS0f5Ww:EE4xQwCTaD4:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=Nm3TKS0f5Ww:EE4xQwCTaD4:V_sGLiPBpWU"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=Nm3TKS0f5Ww:EE4xQwCTaD4:V_sGLiPBpWU" border="0"></img></a>
</div><img src="http://feeds.feedburner.com/~r/reuters/businessNews/~4/Nm3TKS0f5Ww" height="1" width="1" alt=""/>


In [16]:

print (text)

The Boeing Co Starliner spacecraft that failed in its mission to reach the International Space Station was due to barrel down to the Earth's surface early on Sunday with the daunting task of landing safely.
